# ICT 관련 데이터 smartwatch_sleep 로지스틱 회귀분석
스마트 워치를 통해 분당 심박수, 혈중 산소농도, 체온, 손목의 움직임 등 데이터가 수집된다.
이 데이터를 이용하여 뇌파 측정 없이 사용자가 수면상태에 있는지 예측하는 모델을 만들어보자.
sleep변수는 뇌파 측정을 통해 수집된 수면 여부 정보이다.

###  데이터 불러오기


In [2]:
import pandas as pd
import numpy as np

In [8]:
from google.colab import drive
smartwatch_sleep=pd.read_csv('/content/drive/MyDrive/데이터 사이언스 수업자료/7/강의 소스 파일-20230806/data/smartwatch_sleep.csv')

In [7]:
from google.colab import drive
smartwatch_sleep=pd.read_csv('/content/drive/MyDrive/데이터 사이언스 수업자료/7/강의 소스 파일-20230806/data/smartwatch_sleep.csv')

In [12]:
# 샘플 데이터 불러오기
smartwatch_sleep=pd.read_csv("/content/drive/MyDrive/데이터 사이언스 수업자료/7/강의 소스 파일-20230806/data/smartwatch_sleep.csv")
df(smartwatch_sleep)

NameError: ignored

In [ ]:
df.describe()

In [ ]:
X=df.drop(["sleep"],axis=1)
y=df["sleep"]

#### 종속변수와 독립변수의 관계

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
def x_hist_by_y (variable):
    sns.distplot(df[df.sleep == 0][variable], color="blue", label="1")
    sns.distplot(df[df.sleep == 1][variable], color="green", label="0")
    plt.legend(title="target")
    plt.show()

In [ ]:
for i in X.columns :
    x_hist_by_y (i)

#### 다중공선성 분석

In [ ]:
df1 = X.corr()
mask = np.zeros_like(df1, dtype=np.bool)
mask[np.triu_indices_from(mask)]= True
f, ax = plt.subplots(figsize=(11, 9))
ax = sns.heatmap(df1, cmap = 'coolwarm', square = True, mask = mask,
                 vmin = -1, vmax = 1, annot = True, annot_kws = {"size": 10})

#### 다중공선성 분석
* heart rate와 o2는 높은 상관계수를 지님
* movement_intensity와 movement_freq는 높은 상관계수를 지님

### 로지스틱 회귀분석 (Logistic regression)

#### Model 1 : 기본 모델

In [ ]:
import statsmodels.api as sm

model_full = sm.Logit(y, sm.add_constant(X))
result_full = model_full.fit()
result_full.summary()

#### Model 2: 변수 선택후 모델링

In [ ]:
X_reduced= X.drop(["o2","movement_freq"],axis=1)
model_reduced = sm.Logit(y, sm.add_constant(X_reduced))
result_reduced = model_reduced.fit()
result_reduced.summary()

In [ ]:
X_reduced = X.drop(['o2', 'movement_freq','snoring'],axis = 1)
model_reduced = sm.Logit(y, sm.add_constant(X_reduced))
result_reduced = model_reduced.fit(maxiter = 2000,method = 'nm')
result_reduced.summary()

In [ ]:
X_reduced = X.drop(['o2', 'movement_freq', 'snoring', 'body_temp'],axis = 1)
model_reduced = sm.Logit(y, sm.add_constant(X_reduced))
result_reduced = model_reduced.fit(maxiter = 2000,method = 'nm')
result_reduced.summary()

# 머신러닝 방법 비교
## 데이터 구분 : 학습 데이터와 검증데이터
* 전체 데이터를 학습 데이터와 검증 데이터로 50:50으로  구분
* random seed = 1234 / stratify=y (학습과 검증 데이터에서 event 발생 비중을 유지하도록 구분)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
def data_split (x, y) :
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=1234, stratify=y)
    print("Ratio of the event: Training dataset-",round( sum(y_train)/len(y_train)*100,2),"%, Test dataset -" ,round( sum(y_test)/len(y_test)*100,2), "%")
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test=data_split (X,y)

##  Model1 :  로지스틱 회귀모델

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# 변수 선택후 X 변수
X_train_reduced =  X_train.drop(['o2', 'movement_freq', 'snoring', 'body_temp'],axis = 1)
X_test_reduced = X_test.drop(['o2', 'movement_freq', 'snoring', 'body_temp'],axis = 1)

In [ ]:
# training the model & prediction
model_logit = LogisticRegression(random_state=1234, max_iter = 1000, solver='liblinear') # old version(<0.22) default solver 로 통일
fit_logit = model_logit.fit(X_train_reduced, y_train)
y_pred_logit = fit_logit.predict(X_test_reduced)

## Model2 : 신경망
* hidden_layer_sizes=(3,4,5),random_state=1234, max_iter = 1000

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [ ]:
# data scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# training the model & prediction
model_nn = MLPClassifier(hidden_layer_sizes=(3,4,5),random_state=1234, max_iter = 1000)
fit_nn = model_nn.fit(X_train_scaled, y_train)
y_pred_nn =fit_nn.predict(X_test_scaled)

## Model3 : 의사결정나무

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# training the model & prediction
model_tree = DecisionTreeClassifier(random_state = 1234,  max_depth=5)
fit_tree = model_tree.fit(X_train,y_train)
y_pred_tree = fit_tree.predict(X_test)

## Model 4: 랜덤포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model_rf = RandomForestClassifier(random_state=1234)
fit_rf=model_rf.fit(X_train,y_train)
y_pred_rf = fit_rf.predict(X_test)

## Model 5: 서포트 벡터 머신 (SVM)
* linear kernel 이용

In [ ]:
from sklearn import svm

In [ ]:
# linear kernel 이용
model_svm = svm.SVC(kernel='linear', random_state=1234)
fit_svm= model_svm.fit(X_train,y_train)
y_pred_svm = fit_svm.predict(X_test)

###### 모형비교
* 신경망 모델이 AUC = 0.77으로 가장 좋은 성능을 보인다.

In [ ]:
from sklearn.metrics import plot_roc_curve

roc_logit = plot_roc_curve(fit_logit, X_test_reduced, y_test)
roc_nn = plot_roc_curve(fit_nn, X_test_scaled, y_test, ax = roc_logit.ax_)
roc_tree = plot_roc_curve(fit_tree, X_test, y_test, ax = roc_logit.ax_)
roc_rf = plot_roc_curve(fit_rf, X_test, y_test, ax = roc_logit.ax_)
roc_svm = plot_roc_curve(fit_svm, X_test, y_test, ax = roc_logit.ax_)
roc_rf.figure_.suptitle("ROC curve comparison");